<a href="https://colab.research.google.com/github/450586509/practical-ml/blob/master/%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E8%A7%A3%E9%87%8A%E5%BA%93InterpretML_EBM%E4%BE%8B%E5%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install interpret

import pandas as pd
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 5.2MB 7.2MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 1.8MB 40.1MB/s 
     |████████████████████████████████| 3.6MB 40.5MB/s 
     |████████████████████████████████| 5.3MB 22.7MB/s 
     |████████████████████████████████| 471kB 42.0MB/s 
     |████████████████████████████████| 276kB 33.1MB/s 
     |████████████████████████████████| 122kB 42.0MB/s 
     |████████████████████████████████| 788kB 43.4MB/s 
     |████████████████████████████████| 860kB 44.3MB/s 
     |████████████████████████████████| 327kB 43.3MB/s 
     |████████████████████████████████| 1.0MB 45.3MB/s 
     |████████████████████████████████| 3.5MB 43.8MB/s 
     |████████████████████████████████| 194kB 47.1MB/s 
     |████████████████████████████████| 245kB 47.6MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 358kB 44.9MB/s 
     |████████████████████████████████| 358kB 38.8MB/

###数据集介绍
该数据集由Barry Becker从1994人口普查数据库中提取得到。

预测任务是确定一个人年薪是否超过50K。

In [2]:


df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
# df = df.sample(frac=0.1, random_state=1)
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

### 了解数据
每一个特征下面标签的分布

In [7]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
show(hist)

/usr/local/lib/python3.6/dist-packages/interpret/provider/visualize.py:44: UserWarning: Cloud environment detected (['colab', 'ipython']): viz integration is still experimental.
  detected_envs


### 训练模型

In [8]:
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier

ebm = ExplainableBoostingClassifier(random_state=seed)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

ExplainableBoostingClassifier(binning='quantile', early_stopping_rounds=50,
                              early_stopping_tolerance=0.0001,
                              feature_names=['Age', 'WorkClass', 'fnlwgt',
                                             'Education', 'EducationNum',
                                             'MaritalStatus', 'Occupation',
                                             'Relationship', 'Race', 'Gender',
                                             'CapitalGain', 'CapitalLoss',
                                             'HoursPerWeek', 'NativeCountry'],
                              feature_types=['continuous', 'categorical',
                                             'continuous',...gorical',
                                             'continuous', 'categorical',
                                             'categorical', 'categorical',
                                             'categorical', 'categorical',
                                  

### 看看模型学到了什么

In [9]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

### 一个预测是基于什么做出的
给每一个特征打个分

In [16]:
print(y[5:9])

5    0
6    0
7    1
8    1
Name: Income, dtype: int64


In [17]:
ebm_local = ebm.explain_local(X_test[5:9], y_test[5:9], name='EBM')
show(ebm_local)

### 评价EBM的效果
使用ROC和AUC指标

In [15]:
from interpret.perf import ROC

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)